 # Opis zadania
 W tym notatniku stworzymy system analityczno-raportowy, mający wspomagać przyszłe decyzje biznesowe. Dzięki przeprowadzonej wcześniej analizie eksploracyjnej danych, w trakcie spotkania prezentującego wykonaną pracę postanowiono w pierwszym kroku stworzyć następujące wizualizacje danych:
 - Wyświetlenie TOP 10 tras samolotów pod względem najniższego odsetka opóźnień w formie tabeli,
 - Wyświetlenie porównania roku 2019 oraz 2020 w formie wykresu słupkowego:
     - miesiąc do miesiąca,
     - dzień tygodnia do dnia tygodnia,
 - Wyświetlenie danych dzień po dniu w formie szeregu czasowego.

Raport powinien składać się z trzech stron - `TOP report`, `Comparision` i `Day by day reliability`.

Pracując na podstawie wcześniej zdefiniowanych widoków, nie musimy się tutaj martwić o przetwarzanie oraz procesowanie danych. Co więcej, wszystkie niuanse techniczne w postaci liczby wierszy, wykonywanych złączeń, filtracji są przeniesione na bazę danych. Z technicznego punktu widzenia, pomiędzy serwisem a bazą danych dochodzi do przesyłania mniejszej ilości danych.

Dopuszczamy jednak pewne aktualizacje co do ich struktury przykładowo poprzez _pivotowanie_, czyli obranie i doprowadzanie do postaci tej znanej z tabel przestawnych.

> **Uwaga:**  
> Przy pracy nad poprawkami w dashboardzie, pamiętaj, że aby odświeżyć stronę po wprowadzonych zmianach, należy **całość** uruchomić ponownie.

W tej części projektu końcowego wcielasz się ponownie w rolę BI Developera, który ma za zadanie stworzyć dashboard zgodny z wytycznymi biznesowymi dostarczonymi przez klienta. Pamiętaj, że osoba na tym stanowisku często ma kontakt z biznesem więc musi umieć przekazać informację o danych, które zawarła na dashboardzie by te zaangażowały odbiorców.


 Tutaj zaimportuj potrzebne biblioteki

In [108]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import dash
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash import dash_table
from dash.dependencies import Input, Output
import threading
import webbrowser

 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [57]:
username = 'postgres'
password = '9292'
host = 'localhost'
database = 'airlines'
port = 5432

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [58]:
conn = psycopg2.connect(
    dbname=database,
    user=username,
    password=password,
    host=host,
    port=port
)
cursor = conn.cursor()

 # Stworzenie tabeli dla `TOP 10 routes`
 W tym miejscu przygotujemy tabelkę oraz komponent, który zostanie następnie umieszczony w raporcie. Tabela powinna się składać z następujących kolumn:
 - Origin,
 - Destination,
 - Year,
 - Reliability,
 - Rank.

Wartości procentowe zaś powinny być postaci np. 13.87%.

Z tego względu konieczna będzie zmiana nazw kolumn dla wyników raportowania.

Wskazówki:
 - `Python - analiza danych > Dzień 5 - Pandas > Obróbka danych - część 2 > apply`,
 - [How to format percentage in python](https://stackoverflow.com/questions/5306756/how-to-print-a-percentage-value-in-python).

 Tutaj zaczytaj dane do ramki danych `top_routes_df` z widoku `reporting.top_reliability_roads`
 > Jeśli pojawi się komunikat `UserWarning: pandas only support SQLAlchemy`, z naszego punktu widzenia, możemy go śmiało zignorować.

In [59]:
# Zdefiniuj zapytanie SQL, które pobierze dane z widoku
query = "SELECT * FROM reporting.top_reliability_roads"

# Wczytaj dane z widoku do ramki danych
top_routes_df = pd.read_sql_query(query, conn)

# Wyświetl kilka pierwszych wierszy ramki danych
top_routes_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_12748\432420151.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,origin_airport_id,origin_airport_name,dest_airport_id,dest_airport_name,year,cnt,reliability,nb
0,13930,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",12953,"LAGUARDIA AIRPORT, NY US",2019,14702,40.246225,1
1,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",12478,"LAGUARDIA AIRPORT, NY US",2019,12862,38.368838,2
2,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,11722,37.348575,3
3,12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,11718,36.337259,4
4,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",14771,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,14630,35.324675,5


 Tutaj dokonaj aktualizacji formatu liczbowego dla kolumny `reliability`

In [60]:
# Konwersja kolumny "reliability" na format procentowy
top_routes_df["reliability"] = top_routes_df["reliability"].apply(lambda x: f"{x:.0f}%")
top_routes_df

,origin_airport_id,origin_airport_name,dest_airport_id,dest_airport_name,year,cnt,reliability,nb
0,13930,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",12953,"LAGUARDIA AIRPORT, NY US",2019,14702,40%,1
1,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",12478,"LAGUARDIA AIRPORT, NY US",2019,12862,38%,2
2,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,11722,37%,3
3,12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,11718,36%,4
4,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",14771,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,14630,35%,5
5,12953,"LAGUARDIA AIRPORT, NY US",13930,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019,14699,35%,6
6,10721,"BOSTON, MA US",12953,"LAGUARDIA AIRPORT, NY US",2019,10303,34%,7
7,12953,"LAGUARDIA AIRPORT, NY US",10721,"BOSTON, MA US",2019,10309,34%,8
8,14771,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,14614,33%,9
9,12478,"LAGUARDIA AIRPORT, NY US",12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,12804,27%,10


 Tutaj odpowiednio dokonaj zmian nazewnictwa kolumn

In [61]:
# Wybierz interesujące kolumny z top_routes_df
top_routes_df= top_routes_df[['origin_airport_name', 'dest_airport_name', 'year', 'reliability', 'nb']]

# Zmień nazwy kolumn
nowe_nazwy = {'origin_airport_name': 'Origin', 'dest_airport_name': 'Destination', 'nb': 'Rank'}
top_routes_df.rename(columns=nowe_nazwy, inplace=True)
top_routes_df

C:\Users\User\AppData\Local\Temp\ipykernel_12748\1940972234.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Origin,Destination,year,reliability,Rank
0,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US","LAGUARDIA AIRPORT, NY US",2019,40%,1
1,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","LAGUARDIA AIRPORT, NY US",2019,38%,2
2,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,37%,3
3,"MCCARRAN INTERNATIONAL AIRPORT, NV US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,36%,4
4,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,35%,5
5,"LAGUARDIA AIRPORT, NY US","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019,35%,6
6,"BOSTON, MA US","LAGUARDIA AIRPORT, NY US",2019,34%,7
7,"LAGUARDIA AIRPORT, NY US","BOSTON, MA US",2019,34%,8
8,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,33%,9
9,"LAGUARDIA AIRPORT, NY US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,27%,10


In [62]:
top_routes_df = top_routes_df.head(10)
top_routes_df 

,Origin,Destination,year,reliability,Rank
0,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US","LAGUARDIA AIRPORT, NY US",2019,40%,1
1,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","LAGUARDIA AIRPORT, NY US",2019,38%,2
2,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,37%,3
3,"MCCARRAN INTERNATIONAL AIRPORT, NV US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,36%,4
4,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,35%,5
5,"LAGUARDIA AIRPORT, NY US","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019,35%,6
6,"BOSTON, MA US","LAGUARDIA AIRPORT, NY US",2019,34%,7
7,"LAGUARDIA AIRPORT, NY US","BOSTON, MA US",2019,34%,8
8,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,33%,9
9,"LAGUARDIA AIRPORT, NY US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,27%,10


 # Porównanie rok do roku (YoY) 2019 vs. 2020
 W tym miejscu stworzymy wykres oraz komponent, który zostanie następnie umieszczony w raporcie. Wykres powinien przedstawiać porównanie lat 2019 oraz 2020 po miesiącach względem zmiennej `reliability`.

 > Na wykresie chcemy również przedstawić dane, które nie mają porównania tj. od kwietnia do grudnia 2019.

W tym celu wykonamy poniższe zadania:
 - Przekształcimy tabelkę do formy tabeli przestawnej
 - Stworzymy wykres słupkowy porównujący dane rok do roku
 - Opakujemy stworzony wykres w komponent `Dash`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 1 > Budowanie wykresów plotly`,
 - `Python - analiza danych > Dzień 6 - Pandas c.d > Pivot`.

 Tutaj pobierz z widoku `reporting.year_to_year_comparision` dane do ramki `yoy_comparision_df`

In [63]:
# Zdefiniuj zapytanie SQL, które pobierze dane z widoku
query_2 = "SELECT * FROM reporting.year_to_year_comparision"

# Wczytaj dane z widoku do ramki danych
yoy_comparision_df = pd.read_sql_query(query_2, conn)


# Wyświetl kilka pierwszych wierszy ramki danych
yoy_comparision_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_12748\3251398265.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,month,flights_amount,reliability
0,2019,1,89455,33.632553
1,2019,2,81574,39.355677
2,2019,3,93862,34.953442
3,2019,4,91434,34.416082
4,2019,5,96097,37.814916


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `yoy_comparision_to_plot_df`

In [64]:
# Przekształcamy tabelę do formy tabeli przestawnej
yoy_comparision_to_plot_df = yoy_comparision_df.pivot(index='month', columns='year', values='reliability')
yoy_comparision_to_plot_df

year,2019,2020
month,,
1,33.632553,28.086169
2,39.355677,27.835847
3,34.953442,15.810285
4,34.416082,NaN
5,37.814916,NaN
6,42.810802,NaN
7,39.054691,NaN
8,38.250279,NaN
9,29.632968,NaN


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `yoy_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [65]:
# Tworzymy wykres słupkowy
yoy_comparision_fig = px.bar(yoy_comparision_to_plot_df, x=yoy_comparision_to_plot_df.index, y=[2019, 2020], title="Porównanie niezawodności (2019 vs. 2020)")

# Dostosowujemy wykres
yoy_comparision_fig.update_layout(
    xaxis_title="Miesiąc",
    yaxis_title="Niezawodność",
    legend_title="Rok",
    barmode='group'
)

# Aktualizujemy wykres, aby wyświetlał wszystkie miesiące od 1 do 12 na osi x
yoy_comparision_fig.update_xaxes(type='category', tickmode='array', tickvals=list(range(1, 13)), ticktext=['Styczeń', 'Luty', 'Marzec', 'Kwiecień', 'Maj', 'Czerwiec', 'Lipiec', 'Sierpień', 'Wrzesień', 'Październik', 'Listopad', 'Grudzień'])

# Tworzymy aplikację Dash
app = dash.Dash(__name__)

# Definiujemy układ aplikacji
app.layout = html.Div([
    dcc.Graph(figure=yoy_comparision_fig)
])

# Uruchamiamy aplikację
if __name__ == '__main__':
    app.run_server(debug=True)

 # Porównanie dzień tygodnia do dnia tygodnia (WKoWK) 2019 vs. 2020
 Wykonaj analogiczne do poprzedniego kroku, używając jednak w tym momencie danych z widoku `reporting.day_to_day_comparision`

 Tutaj pobierz z widoku `reporting.day_to_day_comparision` dane do ramki `day_to_day_comparision_df`

In [66]:
# Zdefiniuj zapytanie SQL, które pobierze dane z widoku
query_3 = "SELECT * FROM reporting.day_to_day_comparision"

# Wczytaj dane z widoku do ramki danych
day_to_day_comparision_df = pd.read_sql_query(query_3, conn)


# Wyświetl kilka pierwszych wierszy ramki danych
day_to_day_comparision_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_12748\1921198026.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,day_of_week,flights_amount
0,2019,1,167761
1,2019,2,165478
2,2019,3,163889
3,2019,4,166154
4,2019,5,167397


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `day_to_day_comparision_to_plot_df`

In [67]:
# Przekształcamy tabelę do formy tabeli przestawnej
day_to_day_comparision_to_plot_df = day_to_day_comparision_df.pivot(index='day_of_week', columns='year', values='flights_amount')
day_to_day_comparision_to_plot_df

year,2019,2020
day_of_week,,
1,167761,40983
2,165478,39807
3,163889,39581
4,166154,41086
5,167397,41058
6,129485,30545
7,155663,37233


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `day_to_day_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [68]:
# Tworzymy wykres słupkowy
day_to_day_comparision_fig = px.bar(day_to_day_comparision_to_plot_df, x=day_to_day_comparision_to_plot_df.index, y=[2019, 2020], title="Porównanie liczby lotów (2019 vs. 2020)")

# Dostosowujemy wykres
day_to_day_comparision_fig.update_layout(
    xaxis_title="Dzień tygodnia",
    yaxis_title="Liczba lotów",
    legend_title="Rok",
    barmode='group'
)

# Aktualizujemy wykres, aby wyświetlał nazwy dni tygodnia
day_to_day_comparision_fig.update_xaxes(type='category', tickmode='array', tickvals=list(range(1, 8)), ticktext=['Poniedziałek', 'Wtorek', 'Środa', 'Czwartek', 'Piątek', 'Sobota', 'Niedziela'])

# Tworzymy aplikację Dash
app_day_to_day = dash.Dash(__name__)

# Definiujemy układ aplikacji
app_day_to_day.layout = html.Div([
    dcc.Graph(figure=day_to_day_comparision_fig)
])

# Uruchamiamy aplikację
if __name__ == '__main__':
    app_day_to_day.run_server(debug=True)

 # Stworzenie szeregu czasowego
 W tym miejscu stworzymy wykres w formie szeregu czasowego, który zostanie umieszcony w raporcie. Wykres powinien przedstawiać dane w formie szeregu czasowego (dzień po dniu) dla lat 2019 oraz 2020. Dla możliwości identyfikacji poszczególnych lat, zostaliśmy dodatkowo poproszeni o nadanie innego koloru dla roku 2019 oraz 2020.

 > Pamiętaj o stworzeniu wykresu zgodnie z dobrymi praktykami.

 W tym miejscu pobierz dane do ramki `day_by_day_reliability_df`, z widoku `reporting.day_by_day_reliability`.

In [69]:
# Zdefiniuj zapytanie SQL, które pobierze dane z widoku
query_4 = "SELECT * FROM reporting.day_by_day_reliability"

# Wczytaj dane z widoku do ramki danych
day_by_day_reliability_df = pd.read_sql_query(query_4, conn)


# Wyświetl kilka pierwszych wierszy ramki danych
day_by_day_reliability_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_12748\1809363830.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,date,reliability
0,2019-01-01,34.170976
1,2019-01-02,30.959054
2,2019-01-03,32.517764
3,2019-01-04,37.903068
4,2019-01-05,35.035959


 Tutaj stwórz wykres liniowy na podstawie pobranych danych. Wynik zapisz do zmiennej `day_by_day_reliability_fig`.

In [70]:
# Konwertujemy kolumnę 'date' na typ daty
day_by_day_reliability_df['date'] = pd.to_datetime(day_by_day_reliability_df['date'])

# Tworzymy wykres liniowy
fig_day_by_day = px.line(day_by_day_reliability_df, x='date', y='reliability', title="Niezawodność (2019 vs. 2020)")

# Dostosowujemy wykres
fig_day_by_day.update_layout(
    xaxis_title="Data",
    yaxis_title="Niezawodność",
    legend_title="Rok",
    showlegend=True,
    template="plotly",
    hovermode="x unified"
)

# Dodajemy oznaczenie roku 2019 i 2020
fig_day_by_day.add_shape(
    type="line",
    x0="2019-01-01",
    x1="2019-12-31",
    y0=0,
    y1=0,
    line=dict(color="red", width=2),
    name="2019"
)

fig_day_by_day.add_shape(
    type="line",
    x0="2020-01-01",
    x1="2020-12-31",
    y0=0,
    y1=0,
    line=dict(color="blue", width=2),
    name="2020"
)

# Zapisujemy wykres do zmiennej day_by_day_reliability_fig
day_by_day_reliability_fig = fig_day_by_day

day_by_day_reliability_fig.show()

 # Stworzenie layoutów poszczególnych stron

 ## Stworzenie layout dla widoku `TOP 10 routes`
 W tym momencie przekonwertujemy zmienną `top_routes_df` w postać `Dash.DataTable`. Dodatkowo, dla czytelności, dodamy nagłówek opisujący co tabela przedstawia.

 Nagłówek wystylizuj używając poniższych wymagań:
 - tag: `H3`,
 - czcionka: `verdana`,
 - kolor: '#4444`,
 - tekst do wyświetlenia: `TOP 10 reliability routes in 2019 and 2020`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 3 > Dash - datatable`,
 - Dokmentacja metody `Pandas` - [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html).

 Tutaj przygotuj komponent `Dash` `DataTable`, zapisując wynik do zmiennej `top_routes_table`

In [78]:
top_routes_table = dash_table.DataTable(
        id='top-routes-table',
        columns=[
            {'name': 'Origin', 'id': 'Origin'},
            {'name': 'Destination', 'id': 'Destination'},
            {'name': 'Year', 'id': 'year'},
            {'name': 'Reliability', 'id': 'reliability'},
            {'name': 'Rank', 'id': 'Rank'}
        ],
        data=top_routes_layout.to_dict('records')
    )
top_routes_table

DataTable(data=[{'Origin': 'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US', 'Destination': 'LAGUARDIA AIRPORT, NY US', 'year': 2019, 'reliability': '40%', 'Rank': 1}, {'Origin': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'Destination': 'LAGUARDIA AIRPORT, NY US', 'year': 2019, 'reliability': '38%', 'Rank': 2}, {'Origin': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'Destination': 'MCCARRAN INTERNATIONAL AIRPORT, NV US', 'year': 2019, 'reliability': '37%', 'Rank': 3}, {'Origin': 'MCCARRAN INTERNATIONAL AIRPORT, NV US', 'Destination': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'year': 2019, 'reliability': '36%', 'Rank': 4}, {'Origin': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'Destination': 'SAN FRANCISCO INTERNATIONAL AIRPORT, CA US', 'year': 2019, 'reliability': '35%', 'Rank': 5}, {'Origin': 'LAGUARDIA AIRPORT, NY US', 'Destination': 'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US', 'year': 2019, 'reliability': '35%', 'Rank': 6}, {'Origin': 'BOSTON, MA US', 'Destination': 'LAGUARDIA AIRPOR

 Tutaj przygotuj nagłówek dla tabeli, zapisz go pod nazwą `top_routes_page_title`

In [79]:
# Definiowanie nagłówka dla tabeli
top_routes_page_title = html.H3(
            "TOP 10 reliability routes in 2019 and 2020",
            style={"fontFamily": "verdana","color": "#4444"},
)

 Tutaj przygotuj layout dla widoku `TOP 10 routes`. Konfigurację zapisz do zmiennej `top_routes_layout`

In [103]:
# Sample data (replace with your actual data)
top_routes_layout = pd.DataFrame({
    'Origin': [
        'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'MCCARRAN INTERNATIONAL AIRPORT, NV US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'LAGUARDIA AIRPORT, NY US',
        'BOSTON, MA US',
        'LAGUARDIA AIRPORT, NY US',
        'SAN FRANCISCO INTERNATIONAL AIRPORT, CA US',
        'LAGUARDIA AIRPORT, NY US'
    ],
    'Destination': [
        'LAGUARDIA AIRPORT, NY US',
        'LAGUARDIA AIRPORT, NY US',
        'MCCARRAN INTERNATIONAL AIRPORT, NV US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'SAN FRANCISCO INTERNATIONAL AIRPORT, CA US',
        'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US',
        'LAGUARDIA AIRPORT, NY US',
        'BOSTON, MA US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US'
    ],
    'year': [2019] * 10,
    'reliability': ['40%', '38%', '37%', '36%', '35%', '35%', '34%', '34%', '33%', '27%'],
    'Rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
})

# Create a Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = dash.html.Div([
    dash.html.H3("TOP 10 niezawodnych tras w 2019 i 2020"),
    dash_table.DataTable(
        id='top-routes-table',
        columns=[
            {'name': 'Wylot z', 'id': 'Origin'},
            {'name': 'Destynacja', 'id': 'Destination'},
            {'name': 'Rok', 'id': 'year'},
            {'name': 'Niezawodność', 'id': 'reliability'},
            {'name': 'Ranking', 'id': 'Rank'}
        ],
        data=top_routes_layout.to_dict('records')
    )
])


if __name__ == '__main__':
    app.run_server(debug=True)

 ## Stworzenie layout dla widoku `Comparision`
 W tym momencie opakujemy zmienne `yoy_comparision_fig` oraz `day_to_day_comparision_fig` do komponentów `Dash`, a następnie stworzymy odpowiedni `layout` dla strony `comparision`. Wyniki zapiszemy do zmiennej `comparision_layout`.

 Tutaj opakuj `yoy_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `yoy_comparision_component`.

In [81]:
yoy_comparision_component = dcc.Graph(figure=yoy_comparision_fig)

 Tutaj opakuj `day_to_day_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `day_to_day_comparision_component`.

In [82]:
day_to_day_comparision_component = dcc.Graph(figure=day_to_day_comparision_fig)

 Tutaj stwórz odpowiedni `layout` dla widoku `comparision`. Wynik zapisz do zmiennej `comparision_layout`.

In [83]:
# Tworzymy aplikację Dash
app = dash.Dash(__name__)

# Definiujemy układ aplikacji
comparision_layout = html.Div([
    dcc.Graph(figure=yoy_comparision_fig),
    dcc.Graph(figure=day_to_day_comparision_fig)
])

# Przypisujemy układ do aplikacji
app.layout = comparision_layout

# Uruchamiamy aplikację
if __name__ == '__main__':
    app.run_server(debug=True)

 Tutaj stwórz odpowiedni `layout` dla widoku `day by day`. Wynik zapisz do zmiennej `day_by_day_layout`.

In [97]:
# Tworzymy aplikację Dash
app = dash.Dash(__name__)

# Definiujemy układ aplikacji
day_by_day_layout = html.Div([
    dcc.Graph(figure=day_by_day_reliability_fig)
])

# Przypisujemy układ do aplikacji
app.layout = day_by_day_layout

# Uruchamiamy aplikację
if __name__ == '__main__':
    app.run_server(debug=True)

 # Konfigurowanie aplikacji
 Posiadamy już wszystkie niezbędne komponenty do stworzenia oraz uruchomienia aplikacji. Naszym zadaniem w tym miejscu będzie odpowiednie jej skonfigurowanie, aby obsługiwała więcej niż jedną stronę.

 Chcemy, aby możliwe było poruszanie się pomiędzy poniższymi stronami:
 - `TOP report` - strona domyślna,
 - `Comparision` - strona dostępna pod adresem: `comparision_reporting`,
 - `Day by day reporting` - strona dostępna pod adresem: `day_by_day_reporting`

Wskazówki:
 - `Wizualizacja danych > Dzień 4 > Callback context`,
 - `Wizualizacja danych > Dzień 4 > Aplikacja multipage`.
 - Używając komponentu [html.Button](https://community.plotly.com/t/button-with-link/11809), możesz łatwo stworzyć ładniejsze linki do poszczególnych stron.

 Tutaj przygotuj layout aplikacji

In [110]:
# Sample data (replace with your actual data)
top_routes_layout = pd.DataFrame({
    'Origin': [
        'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'MCCARRAN INTERNATIONAL AIRPORT, NV US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'LAGUARDIA AIRPORT, NY US',
        'BOSTON, MA US',
        'LAGUARDIA AIRPORT, NY US',
        'SAN FRANCISCO INTERNATIONAL AIRPORT, CA US',
        'LAGUARDIA AIRPORT, NY US'
    ],
    'Destination': [
        'LAGUARDIA AIRPORT, NY US',
        'LAGUARDIA AIRPORT, NY US',
        'MCCARRAN INTERNATIONAL AIRPORT, NV US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'SAN FRANCISCO INTERNATIONAL AIRPORT, CA US',
        'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US',
        'LAGUARDIA AIRPORT, NY US',
        'BOSTON, MA US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US',
        'LOS ANGELES INTERNATIONAL AIRPORT, CA US'
    ],
    'year': [2019] * 10,
    'reliability': ['40%', '38%', '37%', '36%', '35%', '35%', '34%', '34%', '33%', '27%'],
    'Rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
})

# Create a Dash app
app = dash.Dash(__name__)

# Define the layout for the TOP report page
top_report_layout = dash.html.Div([
    dash.html.H3("TOP 10 niezawodnych tras w 2019 i 2020"),
    dash_table.DataTable(
        id='top-routes-table',
        columns=[
            {'name': 'Wylot z', 'id': 'Origin'},
            {'name': 'Destynacja', 'id': 'Destination'},
            {'name': 'Rok', 'id': 'year'},
            {'name': 'Niezawodność', 'id': 'reliability'},
            {'name': 'Ranking', 'id': 'Rank'}
        ],
        data=top_routes_layout.to_dict('records')
    )
])

# Define the layout for the Comparision page
comparision_layout = html.Div([
    dcc.Graph(figure=yoy_comparision_fig),
    dcc.Graph(figure=day_to_day_comparision_fig)
])

# Define the layout for the Day by day reporting page
day_by_day_layout = html.Div([
    dcc.Graph(figure=day_by_day_reliability_fig)
])

# Define the app's navigation
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        dcc.Link('TOP report', href='/'),
        html.Br(),
        dcc.Link('Comparision', href='/comparision_reporting'),
        html.Br(),
        dcc.Link('Day by day reporting', href='/day_by_day_reporting'),
    ]),
    html.Div(id='page-content'),
])

# Callback to update the page content based on the URL
@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/comparision_reporting':
        return comparision_layout
    elif pathname == '/day_by_day_reporting':
        return day_by_day_layout
    else:
        return top_report_layout

def open_browser():
    webbrowser.open_new('http://127.0.0.1:8050/')

if __name__ == '__main__':
    threading.Timer(1, open_browser).start()
    app.run_server(debug=True)

 # Podsumowanie
 W tym notatniku stworzyliśmy zgodnie z wymaganiami raport do dyspozycji osób decyzyjnych. Kod źródłowy może w tym momencie zostać przekazany dalej, do działu IT, który następnie wdroży rozwiązanie na serwer dostępny dla każdej zainteresowanej osoby. W praktyce oznacza to koniec naszej pracy nad tym zadaniem. Choć warto dodać, że często po udostępnieniu raportu pojawiają się dodatkowe wymagania oraz komentarze ze względu na informacje, które są tam zawarte.

 W kolejnym notatniku podsumujemy sobie cały warsztat.